In [1]:
import datetime
import pandas as pd

from evidently.future.metrics import MeanValue
from evidently.future.datasets import Dataset
from evidently.future.datasets import DataDefinition
from evidently.future.datasets import ColumnInfo
from evidently.future.datasets import ColumnType

from evidently.future.backport import snapshot_v2_to_v1
from evidently.future.backport import SingleValueDashboardPanel
from evidently.future.report import Report
from evidently.future.tests import le
from evidently.ui.workspace import Workspace

def create_snapshot(i):
    df = pd.DataFrame({"col": list(range(i + 5))})
    dataset = Dataset.from_pandas(
        df,
        data_definition=DataDefinition(
            {
                "col": ColumnInfo(ColumnType.Numerical),
            }
        ),
    )
    report = Report([MeanValue(column="col", tests=[le(4)])])
    snapshot_v2 = report.run(dataset, None)

    snapshot_v1 = snapshot_v2_to_v1(snapshot_v2)
    import uuid6

    snapshot_v1.id = uuid6.UUID(int=i, version=7)
    snapshot_v1.timestamp = datetime.datetime.now() - datetime.timedelta(days=1)
    return snapshot_v1

ws = Workspace.create("./workspace")
from evidently.ui.type_aliases import ZERO_UUID

project = ws.get_project(ZERO_UUID)
if project is None:
    from evidently.ui.base import Project

    project = ws.add_project(Project(id=ZERO_UUID, name="test proj"))

    project.dashboard.add_panel(SingleValueDashboardPanel(metric_id="mean:col"))
    project.save()

for i in range(10):
    project.add_snapshot(create_snapshot(i))

In [2]:
# use `evidently ui` to run UI service to see project dashboard.